In [ ]:
!pip install pyspark
!pip install findspark

     |████████████████████████████████| 204.8MB 75kB/s 
     |████████████████████████████████| 204kB 21.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186687 sha256=66091bec7d079a9988d16ef9bdb47239ce2611ede8b22b9879c449c83431839e
  Stored in directory: /root/.cache/pip/wheels/8b/09/da/c1f2859bcc86375dc972c5b6af4881b3603269bcc4c9be5d16
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

In [ ]:
MAX_MEMORY = "5g"

# SparkSession object

spark = SparkSession.builder \
                    .appName('apriori')\
                    .config("spark.executor.memory", MAX_MEMORY) \
                    .config("spark.driver.memory", MAX_MEMORY) \
                    .getOrCreate()

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving store_data.csv to store_data.csv


In [ ]:
import pandas as pd
import io

df = spark.read.csv("store_data.csv")
df

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string]

In [ ]:

def generate_unique_item_set(df):
    # empty dataframe
    total_item_set_df = None
    
    # Iteration over each column - 20 columns
    for col_index in range(20):
        
        # Loading the elements of each column individually
        _c_df = df.select("_c" + str(col_index))
        
        if total_item_set_df is None:
            # None for the first iteration in the loop
            total_item_set_df = _c_df
            
        else:
            # After the first iteration, appending the entries from each column to total_item_set_df
            total_item_set_df = total_item_set_df.union(_c_df)
            
    
    return total_item_set_df.select("_c0").na.drop().rdd.flatMap(remove_duplicate_columns).distinct().toDF()

In [ ]:
# Function to remove the columns with duplicate values

def remove_duplicate_columns(x):
    # Length of the column
    col_len = len(x)
    
    # Empty RDD - set of values
    columns = set()
    
    # Removing any additional spaces from the elements and adding the elements into the column from RDD 'x'
    for col in range(col_len):
        x_col = str(x[col]).strip()
        columns.add(x_col)
    
    # To check if elements are present in the provided dataframe/RDD 
    if len(columns) < col_len:
        return []
    
    # Returning the sorted list of items in each element as tuple
    return [(tuple(sorted(columns)))]

In [ ]:
# Syntax structure: item_sets = function(dataframe)
item_sets = generate_unique_item_set(df)
item_sets.show()

+--------------------+
|                  _1|
+--------------------+
|   whole wheat pasta|
|           asparagus|
|            pancakes|
|         blueberries|
|            zucchini|
|              shrimp|
|             burgers|
|           spaghetti|
|         french wine|
|       strong cheese|
|extra dark chocolate|
|              melons|
|               cream|
|   frozen vegetables|
|           meatballs|
|          energy bar|
|            escalope|
|        energy drink|
|                mint|
|      vegetables mix|
+--------------------+
only showing top 20 rows



In [ ]:
# Give the number of unique items in the transaction dataset
item_sets.count()

119

In [ ]:
def get_freq_item_sets(total_records, candidate_sets_shared, transaction_df_rdd, min_support):    
    #filtered_item_set = transaction_df_rdd.flatMap(lambda x: filter_and_map_transaction(x, candidate_sets_shared)) \
    filtered_item_set = transaction_df_rdd.flatMap(lambda x: filter_and_map_transaction(x, candidate_sets_shared)).reduceByKey(lambda a, b: a+b).map(lambda x: (x[0], x[1]/total_records)).filter(lambda x: x[1]>=min_support)

    return filtered_item_set

In [ ]:
from pyspark.sql.types import StructType, ArrayType, StructField, DoubleType, StringType

def apriori(item_sets, transaction_df_rdd, min_support):
    
    """
    Attributes
    ----------
    item_sets: DataFrame that has all the items present in the transactions
    
    transaction_df_rdd: Transacations in the form of an RDD
    
    min_support: Minimum support threshold
    -----------
    """
    
    total_records = transaction_df_rdd.count()

    freq_item_sets_all_orders = []

    candidate_sets_order_1 = spark.sparkContext.broadcast(item_sets.collect())
    
    frequent_item_sets_order_1 = get_freq_item_sets(total_records, candidate_sets_order_1, transaction_df_rdd, min_support)
    freq_item_sets_all_orders.append(frequent_item_sets_order_1)
    
    frequent_item_sets_order_1_df = frequent_item_sets_order_1.map(lambda x:tuple(x[0])).toDF()
    
    k = 0
    
    while is_freq_item_set_not_empty(freq_item_sets_all_orders[k]):
        
        current_candidate_sets = get_all_possible_candidate_sets(freq_item_sets_all_orders[k], frequent_item_sets_order_1_df)
        current_candidate_sets = spark.sparkContext.broadcast(current_candidate_sets.collect()) 
        current_frequent_item_sets = get_freq_item_sets(total_records, current_candidate_sets, transaction_df_rdd, min_support)
        
        freq_item_sets_all_orders.append(current_frequent_item_sets)
        k += 1
    
    
    return freq_item_sets_all_orders

In [ ]:
# Generate the frequent item set using the apriori function created above.
# Minimum support = 0.01
# Check the format in which each attribute must be provided

transaction_df_rdd = df.rdd
freq_item_sets_all_orders = apriori(item_sets, transaction_df_rdd, 0.01)

print(freq_item_sets_all_orders)

[PythonRDD[403] at RDD at PythonRDD.scala:53, PythonRDD[404] at RDD at PythonRDD.scala:53, PythonRDD[405] at RDD at PythonRDD.scala:53, PythonRDD[406] at RDD at PythonRDD.scala:53]


In [ ]:
freq_item_sets_all_orders[0].take(20)

[(Row(_1='whole wheat pasta'), 0.029462738301559793),
 (Row(_1='pancakes'), 0.09505399280095987),
 (Row(_1='shrimp'), 0.07145713904812692),
 (Row(_1='burgers'), 0.0871883748833489),
 (Row(_1='spaghetti'), 0.17411011865084655),
 (Row(_1='french wine'), 0.022530329289428077),
 (Row(_1='extra dark chocolate'), 0.011998400213304892),
 (Row(_1='melons'), 0.011998400213304892),
 (Row(_1='frozen vegetables'), 0.09532062391681109),
 (Row(_1='meatballs'), 0.020930542594320756),
 (Row(_1='energy bar'), 0.027063058258898813),
 (Row(_1='escalope'), 0.0793227569657379),
 (Row(_1='energy drink'), 0.026663111585121985),
 (Row(_1='mint'), 0.0174643380882549),
 (Row(_1='vegetables mix'), 0.025729902679642713),
 (Row(_1='body spray'), 0.011465137981602452),
 (Row(_1='chocolate'), 0.1638448206905746),
 (Row(_1='butter'), 0.030129316091187842),
 (Row(_1='eggplant'), 0.013198240234635382),
 (Row(_1='fresh tuna'), 0.022263698173576856)]

In [ ]:
freq_item_sets_all_orders[1].take(20)

[(('pancakes', 'shrimp'), 0.010531929076123183),
 (('burgers', 'pancakes'), 0.010531929076123183),
 (('pancakes', 'spaghetti'), 0.025196640447940274),
 (('frozen vegetables', 'pancakes'), 0.013464871350486601),
 (('chocolate', 'pancakes'), 0.01986401813091588),
 (('green tea', 'pancakes'), 0.01639781362485002),
 (('milk', 'pancakes'), 0.01653112918277563),
 (('eggs', 'pancakes'), 0.021730435941874418),
 (('mineral water', 'pancakes'), 0.03372883615517931),
 (('french fries', 'pancakes'), 0.020130649246767097),
 (('ground beef', 'pancakes'), 0.014531395813891481),
 (('cake', 'pancakes'), 0.011865084655379284),
 (('olive oil', 'pancakes'), 0.010798560191974404),
 (('shrimp', 'spaghetti'), 0.021197173710171976),
 (('frozen vegetables', 'shrimp'), 0.01666444474070124),
 (('chocolate', 'shrimp'), 0.017997600319957337),
 (('shrimp', 'tomatoes'), 0.011198506865751233),
 (('green tea', 'shrimp'), 0.011465137981602452),
 (('milk', 'shrimp'), 0.01759765364618051),
 (('eggs', 'shrimp'), 0.0141314

In [ ]:
freq_item_sets_all_orders[2].take(20)

[(('mineral water', 'pancakes', 'spaghetti'), 0.011465137981602452),
 (('frozen vegetables', 'mineral water', 'spaghetti'), 0.011998400213304892),
 (('chocolate', 'milk', 'spaghetti'), 0.010931875749900012),
 (('chocolate', 'eggs', 'spaghetti'), 0.010531929076123183),
 (('chocolate', 'mineral water', 'spaghetti'), 0.01586455139314758),
 (('milk', 'mineral water', 'spaghetti'), 0.01573123583522197),
 (('eggs', 'mineral water', 'spaghetti'), 0.014264764698040262),
 (('french fries', 'mineral water', 'spaghetti'), 0.010131982402346354),
 (('ground beef', 'mineral water', 'spaghetti'), 0.017064391414478068),
 (('mineral water', 'olive oil', 'spaghetti'), 0.010265297960271964),
 (('frozen vegetables', 'milk', 'mineral water'), 0.011065191307825623),
 (('chocolate', 'milk', 'mineral water'), 0.013998133582189041),
 (('chocolate', 'eggs', 'mineral water'), 0.013464871350486601),
 (('chocolate', 'ground beef', 'mineral water'), 0.010931875749900012),
 (('eggs', 'milk', 'mineral water'), 0.0130